<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/8bit_llama2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.0 MB/s eta 0:00:00


In [2]:
!pip install transformers accelerate einops  xformers bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from torch import cuda, bfloat16
from time import time
import transformers
import torch

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-chat-hf", "snapshots", "1")

In [6]:
os.path.exists(model_id)


True

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [9]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
print(model.get_memory_footprint())

7067942912


In [11]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 3.561 sec.


In [12]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=2000,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [20]:
questions = [ "Write me a detailed scenario for a potential Russian invasion of Moldova.",

"Now identify one person in this scenario who would be a potential HUMINT source for the US Intelligence Community and write a short source dossier. Be sure to include their placement and access.",
"Write me three short intelligence information reports that came from that source about various events in the scenario.",
"Add dates and times for the intelligence reports." ]




In [15]:
test_model(tokenizer,
           query_pipeline,
           questions[0])


Test inference: 215.787 sec.
Result: Write me a detailed scenario for a potential Russian invasion of Moldova.
Moldova, a small, landlocked country located between Ukraine and Romania, has been a source of tension between Russia and the West for several years. In recent months, Russia has increased its military presence along Moldova's borders, and there have been reports of Russian-backed separatist groups in the country's breakaway region of Transnistria.
As tensions escalate, the Moldovan government has appealed to the international community for help in defending against a potential Russian invasion. The United States and other Western countries have pledged to provide military aid and support to Moldova, but it remains to be seen whether this will be enough to deter Russia's aggressive actions.
Here is a potential scenario for a Russian invasion of Moldova:
Scenario: Russian Invasion of Moldova
Date: March 20, 2023
Location: Moldova
Objective: To secure control over Moldova's terr

In [17]:
test_model(tokenizer,
           query_pipeline,
           questions[1])


Test inference: 112.794 sec.
Result: Now identify one person in this scenario who would be a potential HUMINT source for the US Intelligence Community and write a short source dossier. Be sure to include their placement and access.

Scenario: A senior engineer at a major technology company is approached by a rival company to steal trade secrets. The engineer is offered a large sum of money and a promotion to a high-level position at the rival company.

Potential HUMINT source: The senior engineer, John Smith.

Source dossier:

Name: John Smith

Placement: Senior engineer at a major technology company

Access: Top-secret research and development projects, including cutting-edge technology and innovative products

Skills: Extensive knowledge of engineering, technology, and product development

Background: John Smith has been with the company for 15 years, starting as an entry-level engineer and working his way up to a senior position. He has a deep understanding of the company's research

In [18]:
test_model(tokenizer,
           query_pipeline,
           questions[2])


Test inference: 85.351 sec.
Result: Write me three short intelligence information reports that came from that source about various events in the scenario.

Intelligence Information Report 1:
Title: Allied Forces to Launch Offensive in Eastern Europe

Date: March 20, 1944

Location: Berlin, Germany

Summary: According to high-ranking sources within the German military, the Allied forces are planning to launch a large-scale offensive in Eastern Europe in the coming weeks. The offensive is expected to target key German military positions and infrastructure, including major cities and transportation hubs. The Allies have assembled a significant force, including ground troops, air support, and naval assets, and are expected to make a major push in the coming weeks.

Intelligence Information Report 2:
Title: Japanese Diplomats in Europe Under Investigation

Date: April 10, 1944

Location: Tokyo, Japan

Summary: Sources within the Japanese government have revealed that several high-ranking di

In [19]:
test_model(tokenizer,
           query_pipeline,
           questions[3])


Test inference: 49.962 sec.
Result: Add dates and times for the intelligence reports.

The intelligence reports are:

1. 2022-01-01 00:00:00 - The enemy is preparing to launch a surprise attack on our position.
2. 2022-01-02 00:00:00:00 - The enemy has launched a surprise attack on our position, but we have repelled the attack.
3. 2022-01-03 00:00:00:00 - The enemy is preparing to launch a counterattack on our position.
4. 2022-01-04 00:00:00:00 - The enemy has launched a counterattack on our position, but we have repelled the attack.
5. 2022-01-05 00:00:00:00 - The enemy is preparing to launch a full-scale attack on our position.

Please provide the dates and times for the intelligence reports.
